In [187]:
from difflib import SequenceMatcher
import sqlite3 as sql
import re

### Fuzzywuzzy documentation

* <https://pypi.org/project/fuzzywuzzy/>
* <https://anaconda.org/conda-forge/fuzzywuzzy>

v. 0.18.0 depuis novembre 2020


In [256]:
from fuzzywuzzy import fuzz

In [261]:
a = 'Johannes Kepler'
b = 'Kepler Johannes'

In [262]:
fuzz.ratio(a, b), fuzz.token_sort_ratio(a,b), fuzz.token_set_ratio(a,b)

(53, 100, 100)

In [15]:
s = SequenceMatcher(None, a, b)
s.ratio()

0.5333333333333333

In [24]:
a[0:3]

'Joh'

In [25]:
def add_underscores(input_string):
    res = '__' + input_string + '__'
    return res

In [27]:
add_underscores(a)

'__Johannes Kepler__'

In [64]:
aa = add_underscores(a)
len(aa)
bb = add_underscores(b)
len(bb), len(aa)

(19, 19)

In [36]:
v = []
i = 0
while i < len(aa)-2:
    ss = aa[i: i+3]
    v.append(ss)cv
    i += 1
print(v)

['__J', '_Jo', 'Joh', 'oha', 'han', 'ann', 'nne', 'nes', 'es ', 's K', ' Ke', 'Kep', 'epl', 'ple', 'ler', 'er_', 'r__']


In [158]:
def trim_strip(s): 
    arr = sorted([a.capitalize().strip(',').strip('-') for a in s.split()])
    return ''.join(sorted([b.upper()  for b in arr]))
    

In [159]:
a = trim_strip('bucci   , - dammi, atto')
a

'ATTOBUCCIDAMMI'

In [162]:
s = SequenceMatcher(None, trim_strip(a), trim_strip(b))
s.ratio()

1.0

In [155]:
ai= sorted([b.upper()  for b in a])
ai

['A', 'A', 'B', 'C', 'C', 'D', 'I', 'I', 'M', 'M', 'O', 'T', 'T', 'U']

In [146]:
c = trim_strip(' - dammi, bucci atto')
ci= sorted([b.upper() for b in c])

['', 'Atto', 'Bucci', 'Dammi']


In [105]:
def compare_trigrams(s1, s2):

    str1= trim_strip(s1)
    str2= trim_strip(s2)
    
    v = []
    i = 0
    while i < len(str1)-2:
        ss = str1[i: i+3]
        v.append(ss)
        i += 1
        v.sort()
    u = []
    i = 0
    while i < len(str2)-2:
        ss = str2[i: i+3]
        u.append(ss)
        i += 1
        u.sort()
        
    print(u)
    print(v)

    if v == u:
        res = 1
    else:
        res = list(set(v)- set(u))
    return res

In [106]:
compare_trigrams(a, b)

['Joh', 'Kep', 'ann', 'epl', 'erJ', 'han', 'ler', 'nes', 'nne', 'oha', 'ple', 'rJo']
['Joh', 'Kep', 'ann', 'epl', 'esK', 'han', 'ler', 'nes', 'nne', 'oha', 'ple', 'sKe']


['esK', 'sKe']

In [77]:
s = 'Kepler ,  joha '

In [78]:
s.split()[0].strip(',')

'Kepler'

In [81]:
''.join([a.strip(',') for a in s.split()])

'Keplerjoha'

In [206]:
s = SequenceMatcher(None, a, b)
s.ratio()

0.5333333333333333

In [46]:
trigrams = lambda a, n : zip(*[a[i:] for i in range(n)])

In [55]:
[(s) for s in trigrams(s,3)]

[('K', 'e', 'p'),
 ('e', 'p', 'l'),
 ('p', 'l', 'e'),
 ('l', 'e', 'r'),
 ('e', 'r', ','),
 ('r', ',', ' '),
 (',', ' ', ' '),
 (' ', ' ', 'j'),
 (' ', 'j', 'o'),
 ('j', 'o', 'h'),
 ('o', 'h', 'a'),
 ('h', 'a', ' ')]

# Interroger la base de données SQLite

In [164]:
db = '../data/generic_database.sqlite'
cn = sql.connect(db)
cn

In [238]:
query = '''
SELECT st1.pk_statement, st2.pk_statement, st1.text_value, st2.numeric_value
FROM statement st1, statement st2
WHERE st1.import_metadata = '20221204_5'
AND st2.import_metadata = '20221210_1'
AND st1.subject_uri = st2.subject_uri
--LIMIT 5
'''

In [239]:
cur = cn.cursor()
cur.execute(query)
rows = cur.fetchall()

bnf = []
for row in rows:
    # print(row)
    bnf.append(row)

In [240]:
bnf[:3], len(bnf)

([(842, 3406, 'François Para Du Phanjas', 1724),
  (843, 3430, 'Jean-Baptiste de Laborde', 1730),
  (844, 2651, 'Johannes Werner', 1468)],
 1079)

In [215]:
stg = "Robert Hamilton (economist)"

In [216]:
stg[stg.find("(")+1:stg.find(")")]

'economist'

In [219]:
stg[0:stg.find("(")]

'Robert Hamilton '

In [247]:
query = '''
SELECT st1.pk_statement, st2.pk_statement, st1.text_value, st2.numeric_value
FROM statement st1, statement st2
WHERE st1.import_metadata = '20221204_3'
AND st2.import_metadata = '20221204_1'
AND st1.subject_uri = st2.subject_uri
AND st2.pk_statement NOT IN (SELECT st2.pk_statement
FROM statement st1, statement st2
WHERE st1.import_metadata = '20221204_4'
AND st2.import_metadata = '20221204_1'
AND st1.subject_uri = st2.subject_uri)
--LIMIT 5
'''

In [248]:
cur = cn.cursor()
cur.execute(query)
rows = cur.fetchall()

dbpedia = []
for row in rows:
    # print(row)
    dbpedia.append(row)

In [249]:
dbpedia[:3], len(dbpedia)

([(295, 3, 'Johannes Engel', 1453),
  (301, 9, 'Matteo Tafuri', 1492),
  (320, 28, 'Rudolph Snellius', 1546)],
 43)

In [159]:
a = trim_strip('bucci   , - dammi, atto')
a

'ATTOBUCCIDAMMI'

In [162]:
s = SequenceMatcher(None, trim_strip(a), trim_strip(b))
s.ratio()

1.0

In [162]:
s = SequenceMatcher(None, trim_strip(a), trim_strip(b))
s.ratio()

1.0

In [255]:
result = []

for i in dbpedia:
    a = i[2]
    for j in bnf:
        b = j[2]
        s = SequenceMatcher(None, trim_strip(a), trim_strip(b))
        r = s.ratio()
        if r > 0.8:
            print(a, b, r)
        

Christopher Heydon Christopher Wren 0.8125
Benjamin Bramer Benjamin Bramer 1.0
Magnus Celsius Magnus Celsius 1.0
Francisco de Borja Garção Stockler Francisco de Borja Garção Stockler 1.0
Luca de Samuele Cagnazzi Luca Cagnazzi de Samuele 1.0
Vincenzo Brunacci Vincenzo Brunacci 1.0


In [ ]:
cn.close()